# Repressilator

Understanding a model from the literature/model repository
https://wwwdev.ebi.ac.uk/biomodels/

* Visualize model
* Create a model SBML report
* Simulate model with tellurium
* Change parameters
* Run stochastic simulations

The model is described at
https://www.ebi.ac.uk/biomodels-main/static-pages.do?page=ModelMonth%2F2006-07

and in the publication [here](../literature/Elowitz2000.pdf)

One of the major goals of Systems Biology is the elucidation of the control logic which determines the behaviour of naturally occuring biological systems[1]. To this end, Systems Biologist often create mathematical models designed to mimic a carefully observed biological system. Traditionally, the modeller aquires data, creates his model and tests the model against the available data. In an interesting take on the conventional way of modelling, Elowitz and Leibler[2] built a mathematical model of transcription regulation describing a cyclic negative-feedback loop made up of three repressor genes and their promoters. They used this model to determine the important parameters of the system and predict the systems behaviour, paying particular attention to parameter values that would cause the system to enter an unstable state leading to oscillatory behaviour. Finally the authors artifically reconstructd the system in E. coli using standard molecular biological approaches. In effect, rather than observing a natural system and explaining it in mathematical terms, the authors create a mathematical model to aid construction of an artificial control circiut. The result is an oscillating network which does not occur in nature, which the authors termed the Repressilator.

![repres1](../images/repres1.png)

The authors created a simple mathematical model of transcription regulation. The mathematical model was composed of six molecular species: three mRNA concentrations and three corresponding repressor protein concentrations. Each species was involved in transcription, translation and degradation reactions. Six coupled first-order differential equations described the dynamic behaviour of the system. Using the model, the authors predicted what parameters the stability of the steady state would be dependent on. In particular, the authors used the model to determine how to induce stable oscillations. Parameters that would favour oscillations were strong promoters, strong repression of transcription, cooperativity of repressor binding and similar lifetimes of mRNA and Proteins. The actual synthetic biological system was constructed from natural components using molecular biological techniques. Two alterations two the natural components were made to bring the system in line with the parameter space which favours oscillations: strong but tightly repressable hybrid promoters, and carboxy-terminal tags for the repressor proteins thus targetting them for protease degradation. A compatible reporter plasmid expressing GFP was also inserted into the system. 

![repres2](../images/repres2.png)

The temporal oscillations of fluorescence occured with a period of approximately 150 minutes, three times as long as the average cell-division time. Additionally, although sibling cells displayed correlated GFP levels for a long period of time after septation, there were variations in both period and amplitude.

Elowitz and Leibler are an excellent example of the use of mathematical models to design artificial control circuits. These can be either used as components for biotechnology and synthetic biology, or used to further study and understand properties of naturally evolved control elements.
Bibliographic References

* [1] Kitano H (2002) Systems Biology: A Brief Overview. Science 295: 1662-1664.
* [2] Elowitz MB, Leibler S (2000) A synthetic oscillatory network of transcriptional regulators. Nature 403: 335-338.

## Model report
HTML overview over the SBML content in a more human readabile manner to understand the model.

In [11]:
import sbmlutils
from sbmlutils.report import sbmlreport
sbmlreport.create_sbml_report("../models/repressilator.xml", out_dir="../models/")

E0: SBML unit consistency (core, L516, code)  
------------------------------------------------------------
[Warning] Missing unit declarations on parameters or literal numbers in expression
In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. 
 The units of the <assignmentRule> <math> expression 'tau_mRNA / log(2)' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.


E1: SBML unit consistency (core, L528, code)  
------------------------------------------------------------
[Warning] Missing unit declarations on parameters or literal numbers in expression
In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in th

## Import model & run simulation
The first simulation experiment with the model reproduces the oscillation behavior of the model shown in Figure~1c of the reference publication. This simulation experiment can be described as:

* Import the repressilator model identified by the Unified Resource Identifier (URI)  `urn:miriam:biomodels.db:BIOMD0000000012`
* Select a deterministic simulation method for the numerical integration.
* Run a uniform time course simulation for 1000~min with an output interval of 1~min.
* Plot the amount of `lacI`, `tetR` and `cI` mRNA and protein against time in a 2D Plot

In [15]:
from IPython.display import display, HTML

import tellurium as te
import pandas as pd

# Load model from biomodels (may not work with https).
r = te.loadSBMLModel("http://biomodels.caltech.edu/download?mid=BIOMD0000000012")

# print the content of the model (antimony)
print(te.sbmlToAntimony(r.getSBML()))

// Created by libAntimony v2.9.4
model *BIOMD0000000012()

  // Compartments and Species:
  compartment cell;
  substanceOnly species PX in cell, PY in cell, PZ in cell, X in cell, Y in cell;
  substanceOnly species Z in cell;

  // Assignment Rules:
  beta := tau_mRNA/tau_prot;
  alpha0 := a0_tr*eff*tau_prot/(ln(2)*KM);
  a0_tr := ps_0*60;
  alpha := a_tr*eff*tau_prot/(ln(2)*KM);
  a_tr := (ps_a - ps_0)*60;
  t_ave := tau_mRNA/ln(2);
  kd_mRNA := ln(2)/tau_mRNA;
  kd_prot := ln(2)/tau_prot;
  k_tl := eff/t_ave;

  // Reactions:
  Reaction1: X => ; kd_mRNA*X;
  Reaction2: Y => ; kd_mRNA*Y;
  Reaction3: Z => ; kd_mRNA*Z;
  Reaction4:  => PX; k_tl*X;
  Reaction5:  => PY; k_tl*Y;
  Reaction6:  => PZ; k_tl*Z;
  Reaction7: PX => ; kd_prot*PX;
  Reaction8: PY => ; kd_prot*PY;
  Reaction9: PZ => ; kd_prot*PZ;
  Reaction10:  => X; a0_tr + a_tr*KM^n/(KM^n + PZ^n);
  Reaction11:  => Y; a0_tr + a_tr*KM^n/(KM^n + PX^n);
  Reaction12:  => Z; a0_tr + a_tr*KM^n/(KM^n + PY^n);

  // Species initializa

In [14]:
# performa the simulation
r.reset()
result = r.simulate(start=0, end=1000, steps=1000)

# which variables are part of the results
print(r.timeCourseSelections)
# plot the results
r.plot(result)
# create a DataFrame from the results
df = pd.DataFrame(result, columns=r.timeCourseSelections)
display(df.head(15))

['time', '[PX]', '[PY]', '[PZ]', '[X]', '[Y]', '[Z]']


,time,[PX],[PY],[PZ],[X],[Y],[Z]
0,0.0,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000
1,1.0,81.440426,188.382020,42.641364,19.903426,30.615526,7.491014
2,2.0,218.538704,358.026782,84.588884,21.233519,23.608864,5.866252
3,3.0,337.622362,469.625864,113.054468,18.544097,17.234851,4.403620
4,4.0,428.938247,536.662776,131.034639,15.559775,12.482022,3.295798
5,5.0,495.496746,572.016986,141.441148,13.017936,9.037664,2.485661
6,6.0,542.465621,585.396059,146.531395,11.033627,6.564237,1.902649
7,7.0,574.777459,583.829764,147.976509,9.561472,4.795373,1.488364
8,8.0,596.569596,572.333053,146.992677,8.518859,3.533290,1.198351
9,9.0,611.138967,554.451402,144.457992,7.823227,2.634105,0.999826


// Created by libAntimony v2.9.4
model *BIOMD0000000012()

  // Compartments and Species:
  compartment cell;
  substanceOnly species PX in cell, PY in cell, PZ in cell, X in cell, Y in cell;
  substanceOnly species Z in cell;

  // Assignment Rules:
  beta := tau_mRNA/tau_prot;
  alpha0 := a0_tr*eff*tau_prot/(ln(2)*KM);
  a0_tr := ps_0*60;
  alpha := a_tr*eff*tau_prot/(ln(2)*KM);
  a_tr := (ps_a - ps_0)*60;
  t_ave := tau_mRNA/ln(2);
  kd_mRNA := ln(2)/tau_mRNA;
  kd_prot := ln(2)/tau_prot;
  k_tl := eff/t_ave;

  // Reactions:
  Reaction1: X => ; kd_mRNA*X;
  Reaction2: Y => ; kd_mRNA*Y;
  Reaction3: Z => ; kd_mRNA*Z;
  Reaction4:  => PX; k_tl*X;
  Reaction5:  => PY; k_tl*Y;
  Reaction6:  => PZ; k_tl*Z;
  Reaction7: PX => ; kd_prot*PX;
  Reaction8: PY => ; kd_prot*PY;
  Reaction9: PZ => ; kd_prot*PZ;
  Reaction10:  => X; a0_tr + a_tr*KM^n/(KM^n + PZ^n);
  Reaction11:  => Y; a0_tr + a_tr*KM^n/(KM^n + PX^n);
  Reaction12:  => Z; a0_tr + a_tr*KM^n/(KM^n + PY^n);

  // Species initializa